In [1]:
#-*- coding:utf-8 -*-

import requests, os
import bs4
from bs4 import BeautifulSoup
from urllib import parse
import re
import lxml
import json
import sys

In [2]:
# return the list of shortcode from html code
def get_shortcode_list(soup):
    soup_script = soup.find_all("script")[3].string
    soup_script = re.split(':|,|\"|\'|\{|\}|\[|\]',soup_script)
    shortcodes = []
    temp = 0
    while 1:
        if 'shortcode' in soup_script[int(temp):]:

            new_shortcode = soup_script.index('shortcode', temp) + 3
            shortcodes.append(soup_script[new_shortcode])
            temp = new_shortcode + 1
        else :
            print("Get ", len(shortcodes), " shortcodes")
            return shortcodes

        
# return the text from html code
def get_text(page):
    script = page.find_all("script")[3].string
    script_split = re.split(':|,|\"|\'|\{|\}|\[|\]',script)  
    text_index = script_split.index('text') + 3
    text = script_split[text_index]  
    
    return text  

# return the user_id and number of likes from html code
def get_id_likes(description):
    data = re.split('on Instagram:', description)

    if '(' in data[0]:
        user_id_index = 6
    else:
        user_id_index = 5

    data = re.split(",|\(|\)|@|Likes|Comments|-",data[0])
    
    likes = data[0]
    user_id = data[user_id_index]
    
    return user_id, likes

# return the text, user_id, number of likes from html code
def get_labels(page):
    for tag in page.select('meta[property*=image]'):
        image_url = tag['content']
            
    for tag in page.select('meta[property*=og:description]'):
        description = tag['content']

    text = get_text(page)
    user_id, likes = get_id_likes(description)
    
    return text, user_id, likes

# save image in page
def save_image(page):
    for tag in page.select('meta[property*=image]'):
        image_url = tag['content']
                
    pic_response = requests.get(image_url).content
    img_name = "./dataset/img/" + word + "_" + shortcode + ".jpg"
    
    with open(img_name, 'wb') as f:
        f.write(pic_response)
    print("file saved: ", img_name)    

In [3]:
word = input("\n** 원하는 태그를 입력하세요: \n>>> ")

# get the url from word(tag)
tag_url = "www.instagram.com/explore/tags/" + word
tag_url = "http://" + parse.quote(tag_url)
print(tag_url)

# get html code
response = requests.get(tag_url).text
soup = BeautifulSoup(response, 'lxml')

# get the list of shortcode
shortcodes = get_shortcode_list(soup)


** 원하는 태그를 입력하세요: 
>>> 경복궁
http://www.instagram.com/explore/tags/%EA%B2%BD%EB%B3%B5%EA%B6%81
Get  77  shortcodes


In [4]:
i = 1
label_data = []

for shortcode in shortcodes:
    print(i, "th ", shortcode)
    i = i + 1  
    
    try:
        # get html code
        page_url = "https://www.instagram.com/p/" + shortcode + "/"
        response = requests.get(page_url).content
        page = BeautifulSoup(response, 'lxml')        
        
        # get label data
        text, user_id, likes = get_labels(page)
        if user_id!=" " and text!="":
            # append new label data 
            output = {"shortcode":shortcode, "label":word, "likes":likes, "user_id":user_id, "text":text}
            label_data.append(output)  

            # save image files
            save_image(page)
            
            print("Okay\n")
        else:
            print("Not okay\n")
    except:
        print("Not okay\n")

1 th  Bp1u7aPACNK
file saved:  ./dataset/img/경복궁_Bp1u7aPACNK.jpg
Okay

2 th  Bp1uo6whe2O
file saved:  ./dataset/img/경복궁_Bp1uo6whe2O.jpg
Okay

3 th  Bp1uQPIh8MX
file saved:  ./dataset/img/경복궁_Bp1uQPIh8MX.jpg
Okay

4 th  Bp1uF_hAWJx
file saved:  ./dataset/img/경복궁_Bp1uF_hAWJx.jpg
Okay

5 th  Bp1t7GnheAw
file saved:  ./dataset/img/경복궁_Bp1t7GnheAw.jpg
Okay

6 th  Bp1t6x_gCOm
file saved:  ./dataset/img/경복궁_Bp1t6x_gCOm.jpg
Okay

7 th  Bp1tyExBRXs
file saved:  ./dataset/img/경복궁_Bp1tyExBRXs.jpg
Okay

8 th  Bp1tmWClkMT
file saved:  ./dataset/img/경복궁_Bp1tmWClkMT.jpg
Okay

9 th  Bp1tgGSHAAc
file saved:  ./dataset/img/경복궁_Bp1tgGSHAAc.jpg
Okay

10 th  Bp1tegbgOrV
file saved:  ./dataset/img/경복궁_Bp1tegbgOrV.jpg
Okay

11 th  Bp1tX94BeT-
file saved:  ./dataset/img/경복궁_Bp1tX94BeT-.jpg
Okay

12 th  Bp1tXmABzqg
file saved:  ./dataset/img/경복궁_Bp1tXmABzqg.jpg
Okay

13 th  Bp1tMRfhaxz
file saved:  ./dataset/img/경복궁_Bp1tMRfhaxz.jpg
Okay

14 th  Bp1tKnFgJ1c
file saved:  ./dataset/img/경복궁_Bp1tKnFgJ1c.jpg
Okay

1

In [5]:
# save labels file
file_name = "./dataset/label.json"
if not os.path.isfile(file_name):
    with open(file_name, mode='w', encoding='UTF-8-sig') as f:
        f.write(json.dumps(label_data, ensure_ascii=False, indent=2))
else:
    with open(file_name) as feedsjson:
        feeds = json.load(feedsjson)

    feeds.append(label_data)
    with open(file_name, mode='w', encoding='UTF-8-sig') as f:
        f.write(json.dumps(feeds, ensure_ascii=False, indent=2))
        
print("file saved: ", file_name)

file saved:  ./dataset/label.json
